## Rekurentné neurónové siete

Pre architektúry rekurentných sietí `keras` poskytuje základné typy ako napr. `LSTM` siete.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(12345)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

### Predspracovanie dát

Dátová množina IMDB obsahuje texty hodnotení filmov, ktoré sú klasifikované podľa ich sentimentu na pozitívne a negatívne. Dáta obsahujú 25 000 príkladov pre trénovanie a 25 000 pre testovanie. Pri načítaní orežeme počet slov na 5 000 najfrekventovanejších slov (ostatné slová v texte budú označené ako neznáme).

In [ ]:
num_words = 5000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=num_words)
print(x_train.shape, x_test.shape)

Dáta sú predspracované ako sekvencie indexov jednotlivých slov v texte. Indexy sú priradené podľa výskytu slov (tzn. index 1 označuje slovo, ktoré sa vyskytlo v textoch najčastejšie). Index 0 reprezentuje neznáme/orezané slová.

In [ ]:
print(x_train[0]) # vypíšeme si prvý trénovací príklad

`keras` vyžaduje, aby boli sekvencie zarovnané na rovnakú dĺžku doplnením 0 indexov (tzv. *padding*). Pre zarovnanie môžete použiť pomocnú funkciu `sequence.pad_sequences`. Štandardne sa 0 hodnoty doplnania na začiatok sekvencie. Maximálnu dĺžku sekvencie nastavíme na 500.

In [ ]:
max_review_length = 500
x_train = sequence.pad_sequences(x_train, maxlen=max_review_length) # zarovnáme trénovacie dáta
x_test = sequence.pad_sequences(x_test, maxlen=max_review_length)   # zarovnáme testovacie dáta

print(x_train[0])      # vypíšeme si zarovnaný prvý trénovací príklad
print(len(x_train[0])) # veľkosť všetkých vektorov reprezentujúich sekvencie bude 500

### Vytvorenie modelu

In [ ]:
# každé slovo budeme reprezentovať ako vektor (tzv. embedding), veľkosť reprezentácie slov nastavíme na 32
# tzn. každý vstupný príklad (text) bude reprezentovaný ako matica 500x32 (maximálna dĺžka sekvencie po zarovnaní x
# veľkosť embedding vektorov)
embedding_length = 32

model = Sequential()
# Embedding vrstva vytvorí prekladovú tabuľku medzi indexom slova a jeho vektorom. Na začiatku sú vektory
# jednotlivých slov reprezentované náhodne, táto vrstva zmení vstupnú zarovnanú sekvenciu indexov slov na maticu
# reprezentujúcu príklad pomocou embedding vektorov jednotlivých slov
model.add(Embedding(num_words, embedding_length, input_length=max_review_length))
# vytvoríme rekurentnú LSTM vrstvu (veľkosť stavu a pamäte je 100)
model.add(LSTM(100))
# výsledná klasifikácia je vypočítaná doprednou vrstvou, ktorá ma na vstupe posledný stav LSTM a na výstupe jeden
# neurón so sigmoidálnou aktivačnou funkciou pre binárnu klasifikáciu
model.add(Dense(1, activation='sigmoid'))

### Trénovanie a validácia na testovacích dátach

In [ ]:
# štandardné nastavenie optimalizačnej metódy pre binárnu klasifikáciu
adam = Adam()
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
# naučíme model, ako validačnú množinu použijeme priamo testovacie dáta
# (tzn. po validácii získame priamo výslednú presnosť modelu)
f = model.fit(x_train, y_train, epochs=3, batch_size=128, validation_data=(x_test, y_test))